In [ ]:
pip install tflearn

In [ ]:
# Libraries needed for NLP
import nltk
nltk.download('punkt')
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

# Libraries needed for Tensorflow processing
import tensorflow as tf
import numpy as np
from tensorflow.python.framework import ops
import random
import json
import tflearn

In [ ]:
# Upload the dataset
from google.colab import files
files.upload()

Saving intents.json to intents.json


{'intents.json': b'\n{"intents": [\n\n\t\t{"tag": "greeting",\n\t\t"patterns": ["Hi", "How are you", "Is anyone there?", "Hello", "Good day","Hey","What\'s up", "Hola", "Howdy"],\n\t\t"responses": ["Hello, thanks for visiting", "Good to see you again", "Hi there, how can I help?"],\n\t\t"context_set": ""\n\t   },\n\t   {"tag": "goodbye",\n\t\t"patterns": ["Bye", "See you later", "Goodbye", "later", "farewell", "bye-bye", "so long"],\n\t\t"responses": ["See you later, thanks for visiting", "Have a nice day", "Bye! Come back again soon."]\n\t   },\n\t   {"tag": "thanks",\n\t\t"patterns": ["Thanks", "Thank you", "That\'s helpful"],\n\t\t"responses": ["Happy to help!", "Any time!", "My pleasure"]\n\t   },\n        {"tag": "vc name",\n        "patterns": ["who is president of csjmu","who is vc of csjmu", "What is the name of vc of csjmu", "who is vice chancellor of csjmu", "who is vice chancellor of chhatrapati sahu ji maharaj university "],\n        "responses": ["Prof. Vinay Kumar Pathak 

In [ ]:
# import our chat-bot intents file
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [ ]:
words = []
classes = []
documents = []
ignore = ['?']
# loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        # add word to the words list
        words.extend(w)
        # add word(s) to documents
        documents.append((w, intent['tag']))
        # add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [ ]:
# Perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))

# remove duplicate classes
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

In [ ]:
# create training data
training = []
output = []
# create an empty array for output
output_empty = [0] * len(classes)

# create training set, bag of words for each sentence
for doc in documents:
    # initialize bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is '1' for current tag and '0' for rest of other tags
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffling features and turning it into np.array
random.shuffle(training)
training = np.array(training)

# creating training lists
train_x = list(training[:,0])
train_y = list(training[:,1])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


In [20]:
def define_model():
  # resetting underlying graph data
  ops.reset_default_graph()

  # Building neural network
  net = tflearn.input_data(shape=[None, len(train_x[0])])
  net = tflearn.fully_connected(net, 10)
  net = tflearn.fully_connected(net, 10)
  net = tflearn.fully_connected(net, len(train_y[0]), activation='softmax')
  net = tflearn.regression(net)

  # Defining model and setting up tensorboard
  model = tflearn.DNN(net, tensorboard_dir='tflearn_logs')
  return model

In [ ]:
model=define_model()
sess = tf.compat.v1.InteractiveSession()

In [ ]:
# Start training
model.fit(train_x, train_y, n_epoch=1000, batch_size=8, show_metric=True)

Training Step: 188999  | total loss: 0.00049 | time: 0.923s
| Adam | epoch: 1000 | loss: 0.00049 - acc: 0.9995 -- iter: 1504/1506
Training Step: 189000  | total loss: 0.00044 | time: 0.928s
| Adam | epoch: 1000 | loss: 0.00044 - acc: 0.9995 -- iter: 1506/1506
--


In [ ]:
# save the model
model.save('model.tflearn')

INFO:tensorflow:/content/model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [ ]:
import pickle
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )


***Use the stored function and data to predict the response***

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd /content/drive/MyDrive/ML_Chatbot/

/content/drive/MyDrive/ML_Chatbot


In [4]:
# restoring all the data structures
import pickle
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [5]:
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [23]:
model=define_model()

# load the saved model
model.load('./model.tflearn')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from /content/drive/MyDrive/ML_Chatbot/model.tflearn


In [24]:
def clean_up_sentence(sentence):
    # tokenizing the pattern
    sentence_words = nltk.word_tokenize(sentence)
    # stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# returning bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=False):
    # tokenizing the pattern
    sentence_words = clean_up_sentence(sentence)
    # generating bag of words
    bag = [0]*len(words)  
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)

    return(np.array(bag))

In [25]:
ERROR_THRESHOLD = 0.30
def classify(sentence):
    # generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    # filter out predictions below a threshold
    results = [[i,r] for i,r in enumerate(results) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], r[1]))
    # return tuple of intent and probability
    return return_list

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    # if we have a classification then find the matching intent tag
    if results:
        # loop as long as there are matches to process
        while results:
            for i in intents['intents']:
                # find a tag matching the first result
                if i['tag'] == results[0][0]:
                    # a random response from the intent
                    return (random.choice(i['responses']))

            results.pop(0)

In [26]:
classify('vc of csjmu')

[('vc name', 0.99989974)]

In [27]:
response('hod of computer science and engineering')

'Dr. Deepak Kumar Verma is hod of computer science and engineering department'

In [28]:
response('how many branches are there in btech')

'BTech in 5 branches namely Computer science and Engineering, Electronics and Communication Engineering, Chemical Engineering, Mechanical Engineering'

In [29]:
response(' hod of ece')

'Er. Niraj kumar is the hod of ece'

In [30]:
response('name ece hod')

'Er. Niraj kumar is the head of department of electronics and communication engineering department'

In [31]:
response('name of director of uiet')

'Dr. Brishti Mitra is the director of uiet'

In [32]:
response('who is vice chancellor of csjmu')

'Prof. Vinay Kumar Pathak'

In [33]:
#Some of other context free responses.
response('courses in csjmu')

'17 Departments,Institute with total of 53 courses, are running in campus'

In [34]:
response('registrar of csjmu')

'Dr. Anil Kumar Yadav'

In [35]:
response('seats in pg diploma')

'there are 60 seats'

In [36]:
response('btech fees')

'83,643INR (per annum)'

In [37]:
response('uiet 4 building')

'University Institute of Engineering and Technology is spread in Six main buildings namely- Academic Building, Computer Science & Engineering, UIET-1, UIET-2, UIET-3, UIET-4, Others'

In [38]:
response('bca syllabus')

'http://csjmu.ac.in/wp-content/uploads/docs/2021/09/bca-syllabus.pdf'

In [39]:
response('hod of mathematics')

'Dr. Varsha Gupta'

In [40]:
response('students in csjmu')

'Dr. Pushpa Mamoria'

In [41]:
response('hod of mechanical')

'Er. Ramendra Singh Niranjan is the head of department of mechanical engineering'

In [42]:
response('hod of chemical')

'Dr. Vinay Kr. Sachan is the hod of chemical engineering'

***ChatBot***

In [56]:
def chat():
    print("The bot is ready to talk!!(Type 'quit' to exit)")
    while True:
        inp = input("\nYou: ")
        if inp.lower() == 'quit':
            break
        result=response(inp)
        if result:
          print("Bot : " + response(inp))
        else:
          print("Bot : Sorry, I am still learning how to answer this type of question.")

chat()

The bot is ready to talk!!(Type 'quit' to exit)

You: Hi
Bot : Hi there, how can I help?

You: who is the vc
Bot : Prof. Vinay Kumar Pathak is the vice chancellor of chhatrapati sahu ji maharaj university

You: hod of chemical
Bot : Dr. Vinay Kr. Sachan is the hod of chemical engineering

You: hod of mechanical
Bot : Er. Ramendra Singh Niranjan is the hod of mechanical

You: uiet director 
Bot : Previously taught as a teacher for several years in the Chemical Engineering Department of UIET

You: uiet director name
Bot : Dr. Brishti Mitra is the name of director of university institute of engineering and technology

You: bye
Bot : Have a nice day

You: quit


***ChatBot With UI***

In [ ]:
!pip install pyngrok==4.1.1

In [ ]:
pip install flask_ngrok

In [53]:
!ngrok authtoken 28W3eip2eyRtSz06buJlOfUkkqB_7pn8geMbRFjRXfi4SAiNr

import numpy as np
from flask_ngrok import run_with_ngrok
from flask import Flask, request, render_template
import pickle

app = Flask(__name__)
run_with_ngrok(app)

@app.route('/')
def home():
    return render_template('index.html')

@app.route('/getprediction',methods=['POST'])
def getprediction():  
    input = [str(x) for x in request.form.values()]
    prediction=response(str(input[0]))

    return render_template('index.html', output='Bot : {}'.format(prediction))
   
if __name__ == '__main__':
    app.run()

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://d61e-35-197-61-242.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [02/May/2022 11:57:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 11:57:44] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/May/2022 11:58:02] "POST /getprediction HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 11:58:18] "POST /getprediction HTTP/1.1" 200 -
127.0.0.1 - - [02/May/2022 11:58:32] "POST /getprediction HTTP/1.1" 200 -
